# Banana Ripeness Classifier


In [1]:
#Import required Modules
import cv2
import numpy as np
import os
from sklearn.preprocessing import LabelEncoder  
from sklearn import svm
from sklearn.metrics import classification_report

# Data Exploration

In [2]:
#Define the metric of Training
def averagecolor(image):
    return np.mean(image, axis=(0, 1))

#Load the dataset
trainX = []
trainY = []

path = "data/"
for label in ("Ripe", "Unripe"):
    print ("Loading training images for the label: "+label)
    
    for filename in os.listdir(path+label+"/"): 
        img = cv2.imread(path+label+"/"+filename)
        img_features = averagecolor(img)
        trainX.append(img_features)
        trainY.append(label)

Loading training images for the label: Ripe
Loading training images for the label: Unripe


# Modelling

In [3]:
#Label Encode the Data
encoder = LabelEncoder()                        
encodedtrainY = encoder.fit_transform(trainY) 

model = svm.SVC(gamma="scale", decision_function_shape='ovr')
model.fit(trainX, encodedtrainY)

SVC()

# Evaluation

In [4]:
# Create the Evaluation Variables
realtestY = np.array(["Unripe","Unripe","Unripe","Unripe","Unripe","Unripe","Unripe","Unripe","Unripe","Unripe","Unripe","Unripe","Unripe","Unripe","Unripe","Ripe","Ripe","Ripe","Ripe","Ripe","Ripe","Ripe","Ripe","Ripe","Ripe","Ripe","Ripe","Ripe","Ripe","Ripe",])
filenames = []
predictedY = []
path = "data/Test/"

def evaluateaccuracy(filenames,predictedY):
    predictedY = np.array(predictedY)
    if (np.sum(realtestY!=predictedY)>0):
        print ("Wrong Predictions: (filename, labelled, predicted) ")
        print (np.dstack([filenames,realtestY,predictedY]).squeeze()[(realtestY!=predictedY)])
    return "Correct :"+ str(np.sum(realtestY==predictedY)) + ". Wrong: "+str(np.sum(realtestY!=predictedY)) + ". Correctly Classified: " + str(np.sum(realtestY==predictedY)*100/len(predictedY))+"%"

for filename in os.listdir(path):
    img = cv2.imread(path+filename)
    img_features = averagecolor(img)
    prediction = model.predict([img_features])[0]
    
    prediction = encoder.inverse_transform([prediction])[0]
    
    print (filename + ": " + prediction)
    filenames.append(filename)
    predictedY.append(prediction)

#Print the Evaluation Report
print ()
print(classification_report(realtestY, predictedY))
print (evaluateaccuracy(filenames,predictedY))

a-unripe (1).jpg: Unripe
a-unripe (10).jpg: Unripe
a-unripe (11).jpg: Unripe
a-unripe (12).jpg: Unripe
a-unripe (13).jpg: Unripe
a-unripe (14).jpg: Unripe
a-unripe (15).jpg: Unripe
a-unripe (2).jpg: Unripe
a-unripe (3).jpg: Unripe
a-unripe (4).jpg: Unripe
a-unripe (5).jpg: Unripe
a-unripe (6).jpg: Unripe
a-unripe (7).jpg: Unripe
a-unripe (8).jpg: Unripe
a-unripe (9).jpg: Unripe
ripe (1).jpg: Ripe
ripe (10).jpg: Ripe
ripe (11).jpg: Ripe
ripe (12).jpg: Ripe
ripe (13).jpg: Ripe
ripe (14).jpg: Unripe
ripe (15).jpg: Ripe
ripe (2).jpg: Ripe
ripe (3).jpg: Ripe
ripe (4).jpg: Ripe
ripe (5).jpg: Ripe
ripe (6).jpg: Ripe
ripe (7).jpg: Ripe
ripe (8).jpg: Unripe
ripe (9).jpg: Ripe

              precision    recall  f1-score   support

        Ripe       1.00      0.87      0.93        15
      Unripe       0.88      1.00      0.94        15

    accuracy                           0.93        30
   macro avg       0.94      0.93      0.93        30
weighted avg       0.94      0.93      0.93        

# Deployment

In [5]:
mode = input("Do you want to use your Webcam Feed [Yes for Webcam and No for Image Upload]: ")
if mode == "No" or mode == "no" or mode == "NO":
    filename = input("Save your file in the 'data' folder and enter the file name here: ")
    filetype = input("Enter the file extension here (ex: .jpg): ")
    #Load the Image to be predicted
    check = cv2.imread("data/"+filename+filetype)
    check2 = averagecolor(check)

    #Predict and display Output
    prediction = model.predict([check2])[0]    
    prediction = encoder.inverse_transform([prediction])[0]
    print ("This banana is " + prediction + ".")
    
elif mode == "Yes" or mode == "yes" or mode == "YES":
    #Getting Webcam Feed
    camera = cv2.VideoCapture(0) 
    while(True):
        ret, frame = camera.read()               
        resized = cv2.resize(frame,(800,600))
            
        #Predicting The Banana
        check2 = averagecolor(resized)
        prediction = model.predict([check2])[0]    
        prediction = encoder.inverse_transform([prediction])[0]
        
        #Displaying the Prediction
        cv2.putText(resized, prediction + " Banana", (650,550), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 0), 2)
        cv2.imshow('Press Spacebar to Exit',resized)  
        
        #Exit Mechanism
        if cv2.waitKey(1) & 0xFF == ord(' '):
            break         
    camera.release()                            
    cv2.destroyAllWindows()   
    

Do you want to use your Webcam Feed [Yes for Webcam and No for Image Upload]: No
Save your file in the 'data' folder and enter the file name here: testpiece 2
Enter the file extension here (ex: .jpg): .jpeg
This banana is Unripe.
